In [8]:
! jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [1]:
! jt -t monokai -T -N -kl

# README


Этот блокнот parsing_alrosa_develop для разработки кода парсера, тестов для переноса в стабильную версию Pycharm
Содержание блоков:
   - BLOCK to/from Pycharm - готовые и разрабатываемые блоки для переноса
   - TESTS тесты
   - DEVELOP - разработка

# BLOCK to/from Pycharm


## stabile

In [1]:
'''
classes_dict.py 

v070922
.print'''
import datetime
import json
import os


class Dicts():
    '''v0003 '''
    folderlog = 'folder/'
    
    
    def __init__(self, dct:dict):
        self.dict = dct
        self.dict['datestr0'] = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        self.dict['datestr'] = None
        if ('name' in dct.keys()) and (dct['name']!=''):
            self.name = dct['name']+ datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        else:
            self.name = 'name'+ datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        self.dict['name']=self.name
        print('установка :', self.name )

    def check(self):
        return True
        
    def save(self):
        if not self.check():
            print('не правильный словарь')
        if self.dict['datestr'] == None:
            self.dict['datestr'] = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        self.filename =  'd_' + self.name + '_' + self.dict['datestr'] + '.json'
        with open(self.folderlog + self.filename, 'w') as writefile:
            json.dump(self.dict, writefile)
            print('dict записан: ', self.filename)
            
    def load(self, filename=None):
        if filename == None:
            files = os.listdir(self.folderlog)
            print([e for e in enumerate(files)])
            i = int(input('введите номер файла: '))
            filename = files[i]
        with open(self.folderlog + files[i], 'r') as writefile:
            self.dict = json.load(writefile)
        print(f'dict загружен :{self.folderlog + filename},\n всего\
        \n загружены атрибуты {self.dict.keys()} \
        ')#{self.dict.values()}')
        print(f'надо определить имя, из какого атрибута взять? \n\
        {[e for e in enumerate(self.dict.keys()) ]}')
        c = int(input('номер: '))
        self.name = self.dict[list(self.dict.keys())[c]]
        print (f'выбрано - {self.name}')
        
        
class Dtask(Dicts):
    '''v0000 Задачи'''
    folderlog = 'tasklogs/'
    def __init__(self, dct):
        super().__init__(dct)

    def check(self):
        return False
   
    def print(self):
        def printdict(d):
            print('{', end='')
            for k in d.keys():
                if type(d[k]) is str:
                    print(f"'{k}': '{d[k]}',")
                else:
                    print(f"'{k}': {d[k]},")
            print('}')
            
            
        print(f'task name: {self.name}')    
        if 'links' in self.dict.keys():
            d = self.dict.copy()
            print('links len = ', len(d.pop('links')),'\n')
            printdict(d)
        else:
            printdict(self.dict)
    



In [2]:
'''
classes_site.py

v220922
'''
import datetime
import json
import os
import re
import requests
from requests import Request, Session
from bs4 import BeautifulSoup
import pandas as pd


class Site():
    '''v0003 for sunlite '''
    folderres = 'results/'
    
    def __init__(self):
        '''parametr: '''
        #self.dict = dct
        print ('create')#self.dict['domain'])
     
    def mysoup(self,urls):
        response = requests.get(urls)
        return BeautifulSoup(response.text, 'lxml')  
  
    def getlinks_sunl(self,urls):
        quotes = self.mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")
        res = [q.get("href") for q in quotes]
        print(len(res), 'links is ready ' , res[0], '...', res[-1])
        return res
        
        
    def genlinks_sunl(self, task):
        '''task: classes_parsing.Dtask'''
        yield task.dict['urls0']
       # 'urlslast':'https://sunlight.net/catalog/diamonds-all/page-104/'
        text = task.dict['urlslast'].strip().split('/')[-2]
        n = int(re.search(r'\d+',text)[0])
        ch = text[re.search(r'\d+',text).span()[0]-1]
        urls = '/'.join(task.dict['urlslast'].strip().split('/')[0:-2])
        for i in range(2,n):
            pages = f'/page{ch}{i}/'
            #pages = f'&page={i}'#без вставки
            yield urls+pages
                  
        
    def getlinks(self, task)->None:#
        '''create list links and modify object task'''
        genlink = self.genlinks_sunl(task)
        links = []
        i = 0
        for l in genlink:
            print(i, l)
            links += self.getlinks_sunl(l)
            i+=1
        task.dict['links'] = links
        


    def parcing(self, task, linknum0=None, linknum=None):
        def savereserve_csv(data):
            datestr = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
            filename = self.folderres + '/_reserve_' + task.dict['name'] + datestr + '.csv'
            data.to_csv(filename)
            return filename

        data = pd.DataFrame(columns=['h1', 'art', 'price', 'price2', 'gold', 'gold2', 'weight', 'gems', 'gems2', 'url'])
        domain = task.dict['domain']
        links = task.dict['links']
        print('парсинг  total ', len(links))
        num0 = 0 if linknum0==None else linknum0
        num = len(links) if linknum==None else linknum
        print(num0,'--->', num)
        for i in range(num0, num):
            try:
                row = i
                urlcard = domain + links[i]
                response = requests.get(urlcard)
                card = BeautifulSoup(response.text, 'lxml')
                # card = mysoup(urlcard)
                data.loc[row, 'url'] = urlcard
                data.loc[row, 'h1'] = card.find("h1").text.strip()
                data.loc[row, 'art'] = card.find('div',
                                                 class_="supreme-product-card__product-article-text").text.strip()
                data.loc[row, 'price'] = card.find('div',
                                                   class_="supreme-product-card__price-discount-price").text.strip().replace(
                    '\u202f', '').replace('\xa0', '')
                data.loc[row, 'price2'] = card.find('div',
                                                    class_="supreme-product-card__price-old").text.strip().replace(
                    '\u202f', '').replace('\xa0', '')
                a = card.find_all('p',
                                  class_="supreme-product-card__description supreme-product-card__description_default")
                data.loc[row, 'gold'] = ';'.join([x.text for x in a])
                data.loc[row, 'd1'] = "&".join(
                    [t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
                data.loc[row, 'd0'] = "&".join(
                    [t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
                # print(i, urlcard)
            except (AttributeError, TypeError) as e:
                print('error', e)
            except IndexError:
                print('IndexError')
            except Exception as e:
                print('ошибка:', e)
                input('жду')
            print(i, urlcard)
            if i % 100 == 0:
                print('saved', i, savereserve_csv(data))

        lastfile = savereserve_csv(data)
        print('saved last', i, lastfile)
        task.dict['datestr'] = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
        task.dict['parcing'] = lastfile
        return data

class Site_alrosa(Site):
    pass

## beta

# TESTS

In [3]:
import datetime
import json
import os
import re
import requests
from requests import Request, Session
from bs4 import BeautifulSoup
import pandas as pd

#from classes_dict import Dtask, Dicts
#from classes_site import Site



## test_sunligt 
при парсинге надо выбрать номер файла и номер поля name 
TODO убрать вводы для тестируемости, добавить тест 0 без выхода в инет

In [4]:
#сбор линков пробный sunlite
#22-09-22 dict записан:  d_SL_diamonds2022-09-22-00-13_2022-09-22-00-13.json
dtask = {'name': 'SL_diamonds',
         'domain': 'https://sunlight.net/',
         # 'links':[],
         'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
         'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-5/',
         'descr': ''}

task = Dtask(dtask)
#task.load()
task.print()
site = Site()
site.getlinks(task)
#site.parcing(task, 0, 1251)

task.save()

установка : SL_diamonds2022-10-12-03-25
task name: SL_diamonds2022-10-12-03-25
{'name': 'SL_diamonds2022-10-12-03-25',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-5/',
'descr': '',
'datestr0': '2022-10-12-03-25',
'datestr': None,
}
create
0 https://sunlight.net/catalog/diamonds-all.html
60 links is ready  /catalog/ring_165495.html ... /catalog/ring_51292.html
1 https://sunlight.net/catalog/diamonds-all/page-2/
60 links is ready  /catalog/earring_146527.html ... /catalog/pendants_56142.html
2 https://sunlight.net/catalog/diamonds-all/page-3/
60 links is ready  /catalog/earring_63402.html ... /catalog/ring_55944.html
3 https://sunlight.net/catalog/diamonds-all/page-4/
60 links is ready  /catalog/ring_233414.html ... /catalog/earring_252294.html
не правильный словарь
dict записан:  d_SL_diamonds2022-10-12-03-25_2022-10-12-03-25.json


In [5]:
#парсинг пробный sunlite
#22-09-22
task = Dtask(dtask)
task.load()
task.print()
site = Site()
#site.getlinks(task)
site.parcing(task, 0, 12)
#task.save()

установка : SL_diamonds2022-10-12-03-252022-10-12-03-25
[(0, 'd_ALROSA_diamonds2022-09-22-01-10_2022-09-22-01-10.json'), (1, 'd_SL_diamonds2022-09-22-00-13_2022-09-22-00-13.json'), (2, 'd_SL_diamonds2022-10-12-02-09_2022-10-12-02-09.json'), (3, 'd_SL_diamonds2022-10-12-03-25_2022-10-12-03-25.json')]
введите номер файла: 3
dict загружен :tasklogs/d_SL_diamonds2022-10-12-03-25_2022-10-12-03-25.json,
 всего        
 загружены атрибуты dict_keys(['name', 'domain', 'urls0', 'urlslast', 'descr', 'datestr0', 'datestr', 'links'])         
надо определить имя, из какого атрибута взять? 
        [(0, 'name'), (1, 'domain'), (2, 'urls0'), (3, 'urlslast'), (4, 'descr'), (5, 'datestr0'), (6, 'datestr'), (7, 'links')]
номер: 0
выбрано - SL_diamonds2022-10-12-03-25
task name: SL_diamonds2022-10-12-03-25
links len =  240 

{'name': 'SL_diamonds2022-10-12-03-25',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamo

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0
0,Золотое кольцо с бриллиантами,Артикул: 165495,4990₽,24950₽,Артикул 165495 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_165495.html,3101-K5R-04&РОССИЯ&Розовое золото&Бриллиант&0....,SERGEY GRIBNYAKOV&Бриллианты. Классика
1,Золотое кольцо «Бриллианты Якутии»,Артикул: 57830,19990₽,66633₽,Артикул 57830 ;Бренд:Бриллианты Якутии ;Кольцо...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_57830.html,0490-K5R-02&РОССИЯ&Розовое золото&Бриллиант&1....,Бриллианты Якутии&Бриллианты Якутии
2,Золотое кольцо с бриллиантами,Артикул: 108790,9990₽,33300₽,Артикул 108790 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_108790.html,70521-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1...,SERGEY GRIBNYAKOV&Бриллианты. Классика
3,Золотое кольцо с бриллиантами,Артикул: 109474,4990₽,24950₽,Артикул 109474 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_109474.html,"3101-K5W-04&РОССИЯ&Белое золото&Бриллиант&3,9 ...",SERGEY GRIBNYAKOV&Бриллианты. Классика
4,Золотое кольцо «Бриллианты Якутии»,Артикул: 146537,12490₽,41633₽,Артикул 146537 ;Бренд:Бриллианты Якутии ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_146537.html,70669-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1...,Бриллианты Якутии&Бриллианты Якутии
5,Золотое кольцо «Бриллианты Якутии»,Артикул: 47720,10990₽,36633₽,Артикул 47720 ;Бренд:Бриллианты Якутии ;Кольцо...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_47720.html,"0496-K5R-01&РОССИЯ&Розовое золото&Бриллиант&1,...",Бриллианты Якутии&Бриллианты Якутии
6,Золотое кольцо с гранатом и бриллиантами,Артикул: 108796,7490₽,37450₽,Артикул 108796 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_108796.html,70536-K5R-01&РОССИЯ&Розовое золото&Бриллиант; ...,SERGEY GRIBNYAKOV&ГРАНАТЫ
7,Золотое кольцо с бриллиантами,Артикул: 152015,4990₽,24950₽,Артикул 152015 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_152015.html,3101-K5Y-04&РОССИЯ&Желтое золото&Бриллиант&0.51 г,SERGEY GRIBNYAKOV&Бриллианты. Классика
8,Золотое кольцо «Бриллианты Якутии»,Артикул: 48271,10990₽,36633₽,Артикул 48271 ;Бренд:Бриллианты Якутии ;Кольцо...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_48271.html,0496-K5W-01&РОССИЯ&Белое золото&Бриллиант&1.08 г,Бриллианты Якутии&Бриллианты Якутии
9,Золотое кольцо с топазами и бриллиантами,Артикул: 283369,9990₽,33300₽,Артикул 283369 ;Бренд:SERGEY GRIBNYAKOV ;Кольц...,NaN,NaN,NaN,NaN,https://sunlight.net//catalog/ring_283369.html,71733-K5R-01&РОССИЯ&Розовое золото&Бриллиант; ...,SERGEY GRIBNYAKOV&ЗОЛОТО. ТОПАЗЫ


## test alrosa 1 (gen links)

In [40]:
#сбор линков alrosa
dtask = {'name': 'ALROSA_diamonds',
         'domain': 'https://sunlight.net/',
         # 'links':[],
         'urls0': 'https://alrosadiamond.ru/diamonds/',
         'urlslast': 'https://alrosadiamond.ru/diamonds/pagen_35/',
         'descr': ''}

task = Dtask(dtask)
#task.load()
#task.print()
#site = Site()
#site.getlinks(task)
#site.parcing(task, 0, 1251)

task.save()

установка : ALROSA_diamonds2022-09-22-01-10
не правильный словарь
dict записан:  d_ALROSA_diamonds2022-09-22-01-10_2022-09-22-01-10.json


# DEVELOP

## get links Alrosa

## 1

In [9]:
urls = 'https://alrosadiamond.ru/diamonds/'

In [10]:
def mysoup(urls):
        response = requests.get(urls)
        return BeautifulSoup(response.text, 'lxml')  

In [11]:
quotes = mysoup(urls)#.find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")

In [ ]:
<a class="product-card__dia21__image-container" href="/diamonds/22084380/" style="max-width: 320px;">
        <img data-src="/assets/build/img/shape_samples/princess.jpg?v=2" class="product-card__dia21__image ls-is-cached lazyloaded" alt="Бриллиант, ID 22084380" title="Бриллиант принцесса 0.76 карат" style="max-width: 100%" src="/assets/build/img/shape_samples/princess.jpg?v=2">
    </a>

In [16]:
quotes = mysoup(urls).find_all('a', class_="product-card__dia21__image-container")

In [17]:
quotes[0]

<a class="product-card__dia21__image-container" href="/diamonds/22084380/" style="max-width: 320px;">
<img alt="Бриллиант, ID 22084380" class="lazyload product-card__dia21__image" data-src="/assets/build/img/shape_samples/princess.jpg?v=2" style="max-width: 100%" title="Бриллиант принцесса 0.76 карат"/>
</a>

In [18]:
res = [q.get("href") for q in quotes]

In [19]:
res

['/diamonds/22084380/',
 '/diamonds/22072730/',
 '/diamonds/21097440/',
 '/diamonds/22007488/',
 '/diamonds/22054481/',
 '/diamonds/11131886/',
 '/diamonds/15084488/',
 '/diamonds/15028853/',
 '/diamonds/15027380/',
 '/diamonds/15021771/',
 '/diamonds/15022784/',
 '/diamonds/15033313/',
 '/diamonds/16029955/',
 '/diamonds/12170661/',
 '/diamonds/16061263/',
 '/diamonds/17028758/',
 '/diamonds/19019585/',
 '/diamonds/18077209/',
 '/diamonds/18071807/',
 '/diamonds/18068508/',
 '/diamonds/18076405/',
 '/diamonds/20118250/',
 '/diamonds/19031096/',
 '/diamonds/18036141/']

#### 2

In [20]:
task.print()

task name: SL_diamonds2022-09-22-00-13
links len =  240 

{'name': 'SL_diamonds2022-09-22-00-13',
'domain': 'https://sunlight.net/',
'urls0': 'https://sunlight.net/catalog/diamonds-all.html',
'urlslast': 'https://sunlight.net/catalog/diamonds-all/page-5/',
'descr': '',
'datestr0': '2022-09-22-00-13',
'datestr': '2022-09-22-00-21',
'parcing': 'results//_reserve_SL_diamonds2022-09-22-00-132022-09-22-00-21.csv',
}


In [45]:
def genlinks_sunl_task(task):#self
    yield task.dict['urls0']
    text = task.dict['urlslast'].strip().split('/')[-2]
    n = int(re.search(r'\d+',text)[0])
    ch = text[re.search(r'\d+',text).span()[0]-1]
    urls = '/'.join(task.dict['urlslast'].strip().split('/')[0:-2])
    for i in range(2,n):
        pages = f'/pagen{ch}{i}/'
            #pages = f'&page={i}'#без вставки
        yield urls+pages

In [46]:
g = genlinks_sunl_task(task)

In [48]:
next(g)

'https://alrosadiamond.ru/diamonds/pagen_2/'

In [ ]:
class Site():
    '''v0003 for sunlite '''
    folderres = 'results/'
    
    def __init__(self):
        '''parametr: '''
        #self.dict = dct
        print ('create')#self.dict['domain'])
     
    def mysoup(self,urls):
        response = requests.get(urls)
        return BeautifulSoup(response.text, 'lxml')  
  
    def getlinks_sunl(self,urls):
        quotes = self.mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")
        res = [q.get("href") for q in quotes]
        print(len(res), 'links is ready ' , res[0], '...', res[-1])
        return res
        
        
    def genlinks_sunl(self, task):
        '''task: classes_parsing.Dtask'''
        yield task.dict['urls0']
       # 'urlslast':'https://sunlight.net/catalog/diamonds-all/page-104/'
        text = task.dict['urlslast'].strip().split('/')[-2]
        n = int(re.search(r'\d+',text)[0])
        ch = text[re.search(r'\d+',text).span()[0]-1]
        urls = '/'.join(task.dict['urlslast'].strip().split('/')[0:-2])
        for i in range(2,n):
            pages = f'/page{ch}{i}/'
            #pages = f'&page={i}'#без вставки
            yield urls+pages
                  
        
    def getlinks(self, task)->None:#
        '''create list links and modify object task'''
        genlink = self.genlinks_sunl(task)
        links = []
        i = 0
        for l in genlink:
            print(i, l)
            links += self.getlinks_sunl(l)
            i+=1
        task.dict['links'] = links
        

